# **Final Project: Ballistic Delivery of Humanitarian Aid:**

https://colab.research.google.com/drive/1D7YGl6aex7qq84Sj-ojRjJAghHC22dVe?usp=sharing

# **Part 1: Numerical Simulation of Projectile Trajectory**

In this program, the code is broken up into 4 parts:

1. Constants, Coordinate Change Functions, and Functions (Gravity, Coriolis, Drag, etc.)
2. The simulation code, including the starting conditions, thrust parameters, etc.
3. Graphing, including plotting the speed, altitude, and distance vs time, as well as all the forces acting on the rocket over the duration of the flight.
4. Function checks, showing how the functions under different conditions perform against experimental data, and showing how they are working correctly and accurately.

The assumtions in the code are given as follows:

1. There are no external forces acting on the rocket aside from gravity, drag, coriolis force, and thrust.
2. The earth is given as an semi-spherioid under the WGS84 model.
3. The rocket was modeled with StarShip like parameters, including the addition of thrust and mass loss due to fuel as variables in this simulation
4. The rocket performs no unusual manuvers (pitchover, full orbits, etc.) and follows a parabolic trajectory with thrust only acting in the direction of movement.

**Libraries and Constants:**

Includes many of the libraries used, and earth constants used for coordinat system transformations and force calculations. All of these values were obtained from google.

In [25]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Earth and universal constants
REarth = 6378137            # Semi-major axis [m] (WGS84)
G = 6.67430e-11             # Gravitational constant [m^3/(kg s^2)]
M_Earth = 5.972e24          # Mass of Earth [kg]
omegaEarth = 7.2921159e-5   # Earth's angular speed [rad/s]

# Atmospheric properties (simple exponential model)
rho0 = 1.225                # Sea-level air density [kg/m^3]
H = 8500                    # Scale height [m]
T0 = 288.15                 # Sea-level temperature [K]
L = 0.0065                  # Temperature lapse rate [K/m]
gamma = 1.4                 # Specific heat ratio
g0 = 9.80665                # Gravity at sea level [m/sec^2]
R = 287.05                  # Specific gas constant for air [J/(kg*K)]

**Coordinate System Conversions:**

Functions to change to and from all of the different coordinate systems used in this project. The functions are named after which coordinate system they change to and from. The coordinate system transformations were taken from the project description.

In [26]:
def geodetic_to_ecef(lat, lon, h):
    """
    Convert geodetic coordinates (lat [°], lon [°], altitude [m])
    to ECEF coordinates [m] using the WGS84 ellipsoidal model.
    """
    lat_rad = np.deg2rad(lat)
    lon_rad = np.deg2rad(lon)
    a = 6378137.0
    e = 0.08181919
    N = a / np.sqrt(1 - e**2 * np.sin(lat_rad)**2)
    X = (N + h) * np.cos(lat_rad) * np.cos(lon_rad)
    Y = (N + h) * np.cos(lat_rad) * np.sin(lon_rad)
    Z = (N * (1 - e**2) + h) * np.sin(lat_rad)
    return np.array([X, Y, Z])

def enu_to_ecef(v_enu, lat, lon):
    """
    Transform a velocity vector from local ENU coordinates to ECEF coordinates.
    v_enu: [v_E, v_N, v_U]
    lat, lon: launch site coordinates in degrees.
    """
    lat_rad = np.deg2rad(lat)
    lon_rad = np.deg2rad(lon)
    R = np.array([
        [-np.sin(lon_rad), -np.sin(lat_rad)*np.cos(lon_rad),  np.cos(lat_rad)*np.cos(lon_rad)],
        [ np.cos(lon_rad), -np.sin(lat_rad)*np.sin(lon_rad),  np.cos(lat_rad)*np.sin(lon_rad)],
        [0.0,              np.cos(lat_rad),                 np.sin(lat_rad)]
    ])
    return R @ np.array(v_enu)

def ecef_to_geodetic(r):
    """
    Convert ECEF coordinates to geodetic coordinates (lat, lon, h).
    Uses an iterative approach.
    """
    x, y, z = r
    a = 6378137.0
    e = 0.08181919
    lon = np.arctan2(y, x)
    r_xy = np.sqrt(x**2 + y**2)
    lat = np.arctan2(z, r_xy)  # initial guess
    for _ in range(5):
        N = a / np.sqrt(1 - e**2 * np.sin(lat)**2)
        h = r_xy/np.cos(lat) - N
        lat = np.arctan2(z, r_xy*(1 - e**2 * N/(N+h)))
    return np.rad2deg(lat), np.rad2deg(lon), h

def ecef_to_enu(v_ecef, lat, lon):
    lat_rad = np.deg2rad(lat)
    lon_rad = np.deg2rad(lon)
    R_inv = np.array([
        [-np.sin(lon_rad),  np.cos(lon_rad),               0],
        [-np.sin(lat_rad)*np.cos(lon_rad), -np.sin(lat_rad)*np.sin(lon_rad), np.cos(lat_rad)],
        [ np.cos(lat_rad)*np.cos(lon_rad),  np.cos(lat_rad)*np.sin(lon_rad), np.sin(lat_rad)]
    ])
    return R_inv @ np.array(v_ecef)


**Gravity, Atmosphere, Coriolis, etc:**

Here are the functions to describe the forces and conditions the rocket is under at every point in time throughout the duration of the fight. These functions are modeled after experimental data for atmospheric codnitions, and general phsyics based formulas for gravity, coriolis, etc. These are all tested at the end of the code after the simulation.

This also includes a method for calculating the mass of the rocket at every point, as we added thrust as a rocket parameter which was not listed within the project description.

In [27]:
layers = [
    (0,     11000,  -0.0065, 288.15, 101325.0),
    (11000, 20000,   0.0,    216.65, 22632.06),
    (20000, 32000,   0.001,  216.65, 5474.89),
    (32000, 47000,   0.0028, 228.65, 868.02),
    (47000, 51000,   0.0,    270.65, 110.91),
    (51000, 71000,  -0.0028, 270.65, 66.94),
    (71000, 84852,  -0.002,  214.65, 3.956)
]

def pressure(h):
    for h_b, h_t, L, T_b, P_b in layers:
        if h < h_t:
            if L == 0:
                return P_b * np.exp(-g0 * (h - h_b) / (R * T_b))
            else:
                T = T_b + L * (h - h_b)
                return P_b * (T / T_b) ** (-g0 / (L * R))
    return 0.3734  # approximate constant above 85 km

def air_density(h):
    T = temperature(h)
    P = pressure(h)
    return P / (R * T)

def temperature(h):
    for h_b, h_t, L, T_b, P_b in layers:
        if h < h_t:
            return T_b + L * (h - h_b) if L != 0 else T_b
    return 186.87  # Value used beyond model range (above 85 km)

def speed_of_sound(h):
    T = max(temperature(h), 1.0)  # prevent sqrt of negative or zero
    return np.sqrt(gamma * R * T)

def gravitational_acceleration(r):
    """
    Compute gravitational acceleration vector [m/s^2] at position r (ECEF).
    """
    r_norm = np.linalg.norm(r)
    g_mag = G * M_Earth / r_norm**2
    return -g_mag * (r / r_norm)

def drag_force(v, h):
    v_mag = np.linalg.norm(v)
    if v_mag < 1e-8:
        return np.zeros(3)

    rho = air_density(h)
    c = speed_of_sound(h)
    M = v_mag / c
    eps = 1e-6  # safety margin to avoid division by 0

    if M < 1:
        Cd = CD0
    else:
        Cd = CD0 + CD1 / np.sqrt(max(M**2 - 1, eps))

    drag_mag = 0.5 * Cd * rho * A * v_mag**2
    drag_vector = -drag_mag * (v / v_mag)
    return drag_vector

def coriolis_acceleration(v):
    omega = np.array([0, 0, omegaEarth])
    return -2 * np.cross(omega, v)

def current_mass(t):
    total_fuel_mass = initial_mass - dry_mass
    max_possible_burn_time = total_fuel_mass / burn_rate

    # Actual burn time is capped by available fuel and user intent
    if user_thrust_duration <= max_possible_burn_time:
        # User ended thrust early: fuel remains
        if t <= user_thrust_duration:
            return initial_mass - burn_rate * t
        else:
            # Burn is over, fuel remains unburned
            burned_fuel = burn_rate * user_thrust_duration
            return dry_mass + (total_fuel_mass - burned_fuel)
    else:
        # User wants longer burn than possible: fuel runs out
        if t <= max_possible_burn_time:
            return initial_mass - burn_rate * t
        else:
            return dry_mass

def thrust_acceleration(v, t):
    m = current_mass(t)
    if t > thrust_duration or np.linalg.norm(v) < 1e-3:
        return np.zeros(3)
    direction = v / np.linalg.norm(v)
    return (thrust_force / m) * direction

def equations_of_motion(t, state):
    """
    Compute the derivative of the state vector for the projectile.
    State vector: [x, y, z, vx, vy, vz] in ECEF.
    Includes:
      - Gravitational acceleration
      - Drag acceleration
      - Coriolis acceleration: -2 ω × v (with ω = [0, 0, ωEarth])
      - Centrifugal acceleration: - ω × (ω × r)
    """
    x, y, z, vx, vy, vz = state
    r = np.array([x, y, z])
    v = np.array([vx, vy, vz])

    _, _, h = ecef_to_geodetic(r)

    mass_current = current_mass(t)

    a_grav = gravitational_acceleration(r)
    a_drag = drag_force(v, h) / mass_current
    a_coriolis = coriolis_acceleration(v)
    a_thrust = thrust_acceleration(v, t)

    a_total = a_grav + a_drag + a_coriolis + a_thrust
    return [vx, vy, vz, a_total[0], a_total[1], a_total[2]]

def hit_ground(t, state):
    """Event function to stop integration when geodetic altitude reaches 0."""
    _, _, h = ecef_to_geodetic(state[:3])
    return h

hit_ground.terminal = True
hit_ground.direction = -1  # Trigger when altitude is decreasing

**Simulation and Graphing:**

In this poriton of the code, the simulate_trajectory method is intruduced, which takes the location and state of the rocket and uses solve_ivp to solve for the values at every location throughout the duration of the flight. Just above includes an event method that calculates the impact location of the rocket, which when triggered stops the solve_ivp method.

Below that is where all of the parameters of the flight are plotted to ensure that the flight makes sence, including the altitude vs. time, range vs time, speed vs time, gravitational force vs time, coriolis force vs time, and drag vs. time. The final landing coordinates are also printed.

Below this, the trajectory of the rocket is projected onto an interactive model of earth, with initial and final impact points plotted, as well as the trajectory above the planet. The launch and impact points correspond with the coordinates on the real planet Earth.

In [28]:
def simulate_trajectory(lat, lon, h0, v_enu, t_span=[0, 50000], max_step=1.0):
    """
    Simulate the projectile trajectory.

    Parameters:
      lat, lon, h0: Launch site geodetic coordinates.
      v_enu: Initial velocity vector in ENU coordinates [m/s], e.g. [v_E, v_N, v_U].
    """
    pos0 = geodetic_to_ecef(lat, lon, h0)
    vel0 = enu_to_ecef(v_enu, lat, lon)
    state0 = np.hstack((pos0, vel0))

    sol = solve_ivp(equations_of_motion, t_span, state0, events=hit_ground, max_step=max_step)
    return sol

def plot_motion_and_forces(sol):
    r = sol.y[:3, :]
    v = sol.y[3:6, :]
    t = sol.t
    alt = np.array([ecef_to_geodetic(r[:, i])[2] for i in range(r.shape[1])])

    # Cumulative arc distance
    stepwise_d = np.linalg.norm(np.diff(r, axis=1), axis=0)
    arc_distance = np.insert(np.cumsum(stepwise_d), 0, 0.0)

    # Speed magnitude
    speed = np.linalg.norm(v, axis=0)

    # Trajectory and speed plots
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.plot(t, alt, 'b-')
    plt.xlabel("Time (s)")
    plt.ylabel("Altitude (m)")
    plt.title("Altitude vs. Time")

    plt.subplot(1, 3, 2)
    plt.plot(t, arc_distance, 'r-')
    plt.ylabel("Cumulative Distance Traveled (m)")
    plt.xlabel("Time (s)")
    plt.title("Total Distance Traveled vs. Time")

    plt.subplot(1, 3, 3)
    plt.plot(t, speed, 'purple')
    plt.xlabel("Time (s)")
    plt.ylabel("Speed (m/s)")
    plt.title("Speed vs. Time")

    plt.tight_layout()
    plt.show()

    # Forces vs time
    Fgrav, Fdrag, Fcoriolis = [], [], []
    omega = np.array([0, 0, omegaEarth])

    for i in range(len(t)):
        state = sol.y[:, i]
        r_vec = state[:3]
        v_vec = state[3:6]
        _, _, h_val = ecef_to_geodetic(r_vec)

        m = current_mass(t[i])
        Fg = m * gravitational_acceleration(r_vec)
        Fd = drag_force(v_vec, h_val)
        Fc = coriolis_acceleration(v_vec) * m

        Fgrav.append(np.linalg.norm(Fg))
        Fdrag.append(np.linalg.norm(Fd))
        Fcoriolis.append(np.linalg.norm(Fc))

    plt.figure(figsize=(12, 8))
    plt.subplot(3, 1, 1)
    plt.plot(t, Fgrav, 'b-', label="Gravitational Force")
    plt.ylabel("Force (N)")
    plt.title("Gravitational Force vs. Time")
    plt.legend()

    plt.subplot(3, 1, 2)
    plt.plot(t, Fdrag, 'r-', label="Drag Force")
    plt.ylabel("Force (N)")
    plt.title("Drag Force vs. Time")
    plt.legend()

    plt.subplot(3, 1, 3)
    plt.plot(t, Fcoriolis, 'g-', label="Coriolis Force")
    plt.xlabel("Time (s)")
    plt.ylabel("Force (N)")
    plt.title("Coriolis Force vs. Time")
    plt.legend()

    plt.tight_layout()
    plt.show()

def check_range(sol):
    """
    Compute and print the total distance traveled along the rocket's trajectory.
    """
    r = sol.y[:3, :]
    # Compute distances between each consecutive state.
    d_total = np.sum(np.linalg.norm(np.diff(r, axis=1), axis=0))
    print(f"Total distance traveled: {d_total:.1f} m")

In [29]:
import plotly.graph_objects as go
from PIL import Image

# Load and process Earth texture
img = Image.open("earth_texture.jpg")  # Download from https://upload.wikimedia.org/wikipedia/commons/8/83/Equirectangular_projection_SW.jpg
#img = Image.open("earth_texture.jpg").transpose(Image.ROTATE_180)
img_array = np.asarray(img)

# Duplicate the first column at the end to fix seam
img_array = np.hstack([img_array, img_array[:, :1, ...]])

# Convert to grayscale for surfacecolor
if img_array.ndim == 3:
    surfacecolor = img_array[:, :, :3].mean(axis=2)
else:
    surfacecolor = img_array

# Create Earth sphere mesh
u = np.linspace(0, 2 * np.pi, surfacecolor.shape[1], endpoint=False)
v = np.linspace(0, np.pi, surfacecolor.shape[0])
U, V = np.meshgrid(u, v)

x_sphere = REarth * np.cos(U) * np.sin(V)
y_sphere = REarth * np.sin(U) * np.sin(V)
z_sphere = REarth * np.cos(V)

def plot_trajectory_on_sphere_interactive_new(sol):
    r = sol.y[:3, :]
    x_traj = r[0, :]
    y_traj = r[1, :]  # Flip Y to fix east-west direction
    z_traj = r[2, :]

    # --- Launch and impact points (on surface) ---
    lat_launch, lon_launch, _ = ecef_to_geodetic(r[:, 0])
    lat_impact, lon_impact, _ = ecef_to_geodetic(r[:, -1])
    lon_launch = np.deg2rad(lon_launch)
    lon_impact = np.deg2rad(lon_impact)
    lat_launch = np.deg2rad(lat_launch)
    lat_impact = np.deg2rad(lat_impact)

    launch_x = REarth * np.cos(lat_launch) * np.cos(lon_launch)
    launch_y = REarth * np.cos(lat_launch) * np.sin(lon_launch)
    launch_z = REarth * np.sin(lat_launch)

    impact_x = REarth * np.cos(lat_impact) * np.cos(lon_impact)
    impact_y = REarth * np.cos(lat_impact) * np.sin(lon_impact)
    impact_z = REarth * np.sin(lat_impact)

    # --- Build Plotly figure ---
    fig = go.Figure()

    # Earth with texture
    fig.add_trace(go.Surface(
        x=x_sphere, y=y_sphere, z=z_sphere,
        surfacecolor=surfacecolor,
        cmin=0, cmax=255,
        colorscale='gray',
        showscale=False,
        name='Earth',
        opacity=1.0
    ))

    # Trajectory line
    fig.add_trace(go.Scatter3d(
        x=x_traj, y=y_traj, z=z_traj,
        mode='lines',
        line=dict(color='orange', width=4),
        name='Trajectory'
    ))

    # Launch marker
    fig.add_trace(go.Scatter3d(
        x=[launch_x], y=[launch_y], z=[launch_z],
        mode='markers',
        marker=dict(size=8, color='green'),
        name='Launch'
    ))

    # Impact marker
    fig.add_trace(go.Scatter3d(
        x=[impact_x], y=[impact_y], z=[impact_z],
        mode='markers',
        marker=dict(size=8, color='red'),
        name='Impact'
    ))

    # Layout
    fig.update_layout(
        title='Interactive 3D Rocket Trajectory on Earth',
        scene=dict(
            xaxis_title='X (m)',
            yaxis_title='Y (m)',
            zaxis_title='Z (m)',
            aspectmode='data'
        ),
        width=900,
        height=900
    )

    fig.show()


**Launch Parameters:**

This cell is where the simulation is actually run. The launch site is speified, as well as the initial speed. The rocket is modeled after SpaceX's Starship rocket, and includes thrust, weight change over time as the rocket expends fuel, and a specified burn time if not all of the fuel is used within it. This is also where the aerodynamic properties were specified, which were found from online sources as well as through experimentation with CD values through the method testing drag below.

In [ ]:
# Launch site parameters (geodetic coordinates)
launch_lat = 45.0        # Latitude [deg]
launch_lon = 90.0         # Longitude [deg]
launch_alt = 0.0         # Altitude [m]

# Launch parameters
V_Up = 500.0
V_East = 100.0
V_North = 0.0
v_enu = [V_East, V_North, V_Up]

thrust_force = 71171545.6    # N
initial_mass = 5000000       # kg (wet mass: fuel + dry)
dry_mass = 500000            # kg (non-burnable structure)
burn_rate = 10000        # kg/s
user_thrust_duration = 300  # s (user-requested)
max_burn_time = (initial_mass - dry_mass) / burn_rate
thrust_duration = min(user_thrust_duration, max_burn_time)
# Projectile properties (mass updated to 10,000 kg)
A = 1.5                 # Cross-sectional area [m^2]
CD0 = 0.295
CD1 = 0.2


# Run the simulation with the above parameters.
solution = simulate_trajectory(
    launch_lat, launch_lon, launch_alt, v_enu
)

lat_deg, lon_deg, h = ecef_to_geodetic(solution.y[:3, -1])

# Flip longitude to match visual globe rendering
lon_deg = (lon_deg - 180) % 360

print("Impact geodetic coordinates (lat, lon, alt):", (lat_deg, lon_deg, h))


# Plot the trajectory.
plot_motion_and_forces(solution)

# Check and print the range.
check_range(solution)

plot_trajectory_on_sphere_interactive_new(solution)

These results make sence given the initial rocket parameters given. The altitude is given as a parabola, starting and ending from 0 meters of altitude. The speed increases almost quadratically with the thrust, until it stops when the burn concludes, speed up even more as the rocket falls back down to earth. The distance also makes sence, as it also increases quadratically with the acceleration of the rocket. The gravitational force graph makes sence, as it is parabolic with higher gravitational force the lower the rocket is. It also starts much higher than it ends due to the mass change of the rocket through the use of fuel. The drag also makes sence, with the value not being very high on the acent due to the relatively low speed at the beginning when the air is dense, only becoming large when the rocket is traveling at its fastest speed coming back down towards earth when the atmosphere is thicker. The coriolis graph also makes sence, as it is dependent on launch direction and mass, as will be shown below.

The globe also shows the path the rocket takes, which corresponds perfectly with the listed launch and landing coordinates. The trajectory is also plotted, showing the initially large slope in the ascent, and how the rocket falls back down the earth.

# **Part 1 Code Method Verifications**

**Verification of Code - Coordinate System Conversions**

Here, a given set of coordinates and initial launch velocities are given and changed too and from each coordinate system. The output shows how these values stay then same when changed too and from each coordinate system, showing their accuracy.

In [ ]:
def test_coordinate_conversions():
    lat, lon, h = 45.0, 90.0, 1000.0  # example point
    v_enu = np.array([500.0, 0.0, 100.0])  # east, north, up

    print("=== Coordinate System Conversion Test ===")
    print(f"Original Geodetic Coordinates: lat={lat}, lon={lon}, h={h}")

    ecef = geodetic_to_ecef(lat, lon, h)
    print(f"Converted to ECEF: X={ecef[0]:.3f}, Y={ecef[1]:.3f}, Z={ecef[2]:.3f}")

    lat2, lon2, h2 = ecef_to_geodetic(ecef)
    print(f"Back to Geodetic: lat={lat2:.6f}, lon={lon2:.6f}, h={h2:.3f}")

    print("\nOriginal ENU velocity vector:", v_enu)
    v_ecef = enu_to_ecef(v_enu, lat, lon)
    print("Converted velocity to ECEF:", v_ecef)

    v_enu_back = ecef_to_enu(v_ecef, lat, lon)
    print("Back to ENU velocity vector:", v_enu_back)

test_coordinate_conversions()

**Verification of Code - Gravity**

This method is verified by plotting gravitiational acceleration vs a given set of altitudes. When the altitude os close to 0 it is given as just about 9.81 m/sec^2, which corresponds to the known value of g. This graph shows a linear negative trend with altitude, which makes sence and corresponds to the formula being dependent on radius.

In [ ]:
def test_gravitational_acceleration():
    altitudes = np.linspace(0, 200000, 100)
    gravities = [np.linalg.norm(gravitational_acceleration(np.array([0, 0, REarth + h]))) for h in altitudes]
    plt.figure()
    plt.plot(altitudes, gravities)
    plt.title("Gravitational Acceleration vs Altitude")
    plt.xlabel("Altitude [m]")
    plt.ylabel("g [m/s²]")
    plt.grid(True)
    plt.show()

test_gravitational_acceleration()

**Verification of Code - Atmospheric Qualities (Temperature, Density, Pressure, and Speed of Sound)**

In this test, we plot experimental data for altitude, temperature, density, and speed of sound vs the functions. We see that the functions align exactly with the experimental results because the functions are based on the same experimental results, making them as accurate as they can be.

In [ ]:
# Reference atmospheric data for comparison (simplified sample from U.S. Standard Atmosphere 1976)
reference_altitudes = np.array([0, 1000, 5000, 10000, 15000, 20000, 25000, 30000, 40000, 50000, 60000, 70000, 80000])
reference_temperature = np.array([288.15, 281.65, 255.65, 223.15, 216.65, 216.65, 221.65, 226.65, 250.65, 270.65, 247.02, 216.65, 196.65])
reference_density = np.array([1.2250, 1.1120, 0.7364, 0.4135, 0.1948, 0.08891, 0.04008, 0.01841, 0.003996, 0.001027, 0.0003097, 8.283e-5, 1.846e-5])
reference_speed_of_sound = np.sqrt(gamma * R * reference_temperature)

def test_atmospheric_properties():
    altitudes = np.linspace(0, 85000, 300)
    T_vals = [temperature(h) for h in altitudes]
    rho_vals = [air_density(h) for h in altitudes]
    a_vals = [speed_of_sound(h) for h in altitudes]

    fig, axs = plt.subplots(1, 3, figsize=(18, 5))

    axs[0].plot(altitudes, T_vals, label='Model')
    axs[0].plot(reference_altitudes, reference_temperature, 'ro', label='Reference')
    axs[0].set_title("Temperature vs Altitude")
    axs[0].set_ylabel("T [K]")

    axs[1].plot(altitudes, rho_vals, label='Model')
    axs[1].plot(reference_altitudes, reference_density, 'ro', label='Reference')
    axs[1].set_title("Air Density vs Altitude")
    axs[1].set_ylabel("ρ [kg/m³]")

    axs[2].plot(altitudes, a_vals, label='Model')
    axs[2].plot(reference_altitudes, reference_speed_of_sound, 'ro', label='Reference')
    axs[2].set_title("Speed of Sound vs Altitude")
    axs[2].set_ylabel("a [m/s]")

    for ax in axs:
        ax.set_xlabel("Altitude [m]")
        ax.grid(True)
        ax.legend()

    plt.tight_layout()
    plt.show()

test_atmospheric_properties()


**Verification of Code - Coriolis**

In this function, the Coriolis force function is tested. The function is plotted under different launch conditions, showing the coriolis forces affect, and how it only plays a factor when the rocket has an east/west trajectory component.

In [ ]:
# Debug and validate coriolis_acceleration function

# We will sweep velocities in northward direction and plot the resulting Coriolis acceleration
northward_speeds = np.linspace(0, 2000, 200)
coriolis_accels = [coriolis_acceleration(np.array([0, v, 0])) for v in northward_speeds]
coriolis_mags = [np.linalg.norm(a) for a in coriolis_accels]

# Also extract component-wise acceleration to verify direction
ax = [a[0] for a in coriolis_accels]
ay = [a[1] for a in coriolis_accels]
az = [a[2] for a in coriolis_accels]

plt.figure(figsize=(12, 6))
plt.plot(northward_speeds, coriolis_mags, label='Total Coriolis Acceleration Magnitude')
plt.plot(northward_speeds, ax, '--', label='X (East) Component')
plt.plot(northward_speeds, ay, '--', label='Y (North) Component')
plt.plot(northward_speeds, az, '--', label='Z (Up) Component')
plt.title("Coriolis Acceleration vs Northward Velocity")
plt.xlabel("Northward Velocity [m/s]")
plt.ylabel("Coriolis Acceleration [m/s²]")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


**Verification of Code - Drag**

In this function, the drag function is tested with respect of a Lapua Scenar 0.338 bullet, in which the data was found from an online source. Experimental CD0 and CD1 values were generated for the rocket, and they were both plotted with respect to speed. The corilation betwen the experimental results and the function is quite strong, and the jump in the function graph is attributed to the transition to supersonic speed, which was expected and shows how the function is behaving normally.

In [ ]:
CD0 = 0.295
CD1 = 0.2
A = 1.5
def experimental_drag_forces_updated(v_range, h):
    result = []
    for v in v_range:
        F = drag_force(np.array([0, v, 0]), h)
        result.append(np.linalg.norm(F))
    return np.array(result)

# Experimental drag coefficient data from ballistic testing (Lapua Scenar .338 bullet)
exp_mach = np.array([0.8, 0.9, 1.0, 1.2, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0, 5.5])
exp_cd = np.array([0.236, 0.306, 0.334, 0.345, 0.348, 0.343, 0.336, 0.328, 0.321, 0.304, 0.292, 0.282, 0.270])

# Convert Mach numbers to speeds at 10 km altitude
h_exp = 10000
c_exp = speed_of_sound(h_exp)
exp_speed = exp_mach * c_exp

# Compute experimental drag force values
exp_force = 0.5 * exp_cd * air_density(h_exp) * A * exp_speed**2

# Recalculate model drag force for comparison
v_range = np.linspace(50, 2000, 200)
model_drag = [np.linalg.norm(drag_force(np.array([v, 0, 0]), h_exp)) for v in v_range]

# Plot
plt.figure(figsize=(10, 5))
plt.plot(v_range, model_drag, label="Modeled Drag Force")
plt.plot(exp_speed, exp_force, 'ro', label="Experimental Data")
plt.title("Modeled vs Experimental Drag Force at 10 km Altitude")
plt.xlabel("Speed [m/s]")
plt.ylabel("Drag Force [N]")
plt.legend()
plt.grid(True)
plt.show()


**Verification of Code - Thrust**

This function tests adding thrust to the rocket, showing the force that is acting on the rocket during a flight and after the trust is cutoff. The drop in acceleration drops to 0 when the cutoff is applied as expected, and the linear increase in thrust acceleration is attributed to the loss of mass as fuel is burned, also supporting the functions validity.

In [ ]:
# Example settings
test_velocity = np.array([100, 100, 100])  # m/s in arbitrary direction
test_times = np.linspace(0, 200, 300)      # covers before, during, and after thrust duration

# Constants
thrust_force = 7.1e7       # Newtons
thrust_duration = 150      # seconds
mass = 5000000             # Starship-like rocket

# Evaluate and plot thrust acceleration over time
a_thrusts = [thrust_acceleration(test_velocity, t) for t in test_times]
magnitudes = [np.linalg.norm(a) for a in a_thrusts]

plt.figure(figsize=(8, 4))
plt.plot(test_times, magnitudes, label='Thrust Acceleration Magnitude')
plt.axvline(thrust_duration, color='r', linestyle='--', label='Thrust Cutoff')
plt.title("Thrust Acceleration vs Time")
plt.xlabel("Time (s)")
plt.ylabel("Thrust Acceleration [m/s²]")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

**Verification of Code - Changing Mass**

This function shows how the mass of the rocket changes over time, and tests different thrust shutoff conditions with different amounts of mass remaining on the rocket. At different points in time the rocket thrust is shut off, stopping the loss in mass, as expected.

In [ ]:
# Diagnostic function to test current_mass over time, including limits
# Updated current_mass to correctly handle excess fuel when thrust ends early
def test_current_mass_dual():
    times = np.linspace(0, 400, 500)

    # Scenario 1: User-defined thrust duration exceeds fuel capacity
    long_burn_duration = 300
    total_fuel_mass = initial_mass - dry_mass
    max_possible_burn_time = total_fuel_mass / burn_rate
    effective_cutoff_long = min(long_burn_duration, max_possible_burn_time)

    def current_mass_long(t):
        if long_burn_duration <= max_possible_burn_time:
            if t <= long_burn_duration:
                return initial_mass - burn_rate * t
            else:
                burned = burn_rate * long_burn_duration
                return dry_mass + (total_fuel_mass - burned)
        else:
            if t <= max_possible_burn_time:
                return initial_mass - burn_rate * t
            else:
                return dry_mass

    masses_long = [current_mass_long(t) for t in times]

    # Scenario 2: User-defined thrust duration shorter than fuel capacity
    short_burn_duration = 100
    effective_cutoff_short = min(short_burn_duration, max_possible_burn_time)

    def current_mass_short(t):
        if short_burn_duration <= max_possible_burn_time:
            if t <= short_burn_duration:
                return initial_mass - burn_rate * t
            else:
                burned = burn_rate * short_burn_duration
                return dry_mass + (total_fuel_mass - burned)
        else:
            if t <= max_possible_burn_time:
                return initial_mass - burn_rate * t
            else:
                return dry_mass

    masses_short = [current_mass_short(t) for t in times]

    # Plot both scenarios
    plt.figure(figsize=(10, 5))
    plt.plot(times, masses_long, label="Mass (Long Burn Request)", color='red')
    plt.axvline(effective_cutoff_long, linestyle='--', color='red', label="Effective Cutoff (Long Burn)")

    plt.plot(times, masses_short, label="Mass (Short Burn Request)", color='blue')
    plt.axvline(effective_cutoff_short, linestyle='--', color='blue', label="Effective Cutoff (Short Burn)")

    plt.axhline(dry_mass, linestyle=':', color='gray', label="Dry Mass")
    plt.xlabel("Time (s)")
    plt.ylabel("Mass (kg)")
    plt.title("Rocket Mass vs Time (Dual Thrust Duration Test)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# Run the dual scenario test
test_current_mass_dual()


# **Part 2: Flight Path Optimization**

In this section, initial trajectory parameters (v_0, theta, psi) are optimized for minimum initial speed v_0 with impact coordinate constraints of +/- 0.1 degrees of geodetic distance from given target coordinates. Using the optimization function and the determined optimal lunch assignments, the trajectory parameters for launches to targets in Table 1 from launch sites in Table 2 are optimized. Assumptions made for the optimization problem include that launch distance is proportional to launch time - therefore target assignments were based on distance between targets. Furthermore, optimization constraints were satisfied when impact coordinates were within +/- 0.1 degrees of geodetic distance which translates to about 111 kilometers.

This cell defines the optimization of initial trajectory parameters of a single launch

In [ ]:
import scipy


def objective_func(x):

  v0, theta, psi = x


  v_E = v0 * np.cos(np.radians(theta)) * np.sin(np.radians(psi))
  v_N = v0 * np.cos(np.radians(theta)) * np.cos(np.radians(psi))
  v_U = v0 * np.sin(np.radians(theta))
  v_enu = [v_E,v_N,v_U]
  sol = simulate_trajectory(laun_lat, laun_lon, laun_h, v_enu,t_span=[0, 10000])

  final_lat, final_lon, _ = ecef_to_geodetic(sol.y[:3, -1])

  if len(sol.t_events[0])>0:
    #return arc_distance
    #return d_total
    #print(f"v0: {v0}, theta: {theta}, impact time: {sol.t_events[0]}")
    return v0
    #float(v0) + float(np.sqrt((abs(final_lat - lat_target))**2 + (abs(final_lon - lon_target))**2))
  else: return np.inf


def constraint_fxn(x, launch_lat, launch_lon, h0, target_lat, target_lon):
    v0, theta, psi = x


    # Convert launch angles to ENU velocity
    v_E = v0 * np.cos(np.radians(theta)) * np.sin(np.radians(psi))
    v_N = v0 * np.cos(np.radians(theta)) * np.cos(np.radians(psi))
    v_U = v0 * np.sin(np.radians(theta))
    v_enu = [v_E, v_N, v_U]

    # Simulate trajectory
    sol = simulate_trajectory(launch_lat, launch_lon, h0, v_enu)

    # Final position in ECEF, then convert to lat/lon/alt
    final_lat, final_lon, _ = ecef_to_geodetic(sol.y[:3, -1])

    # Constraint: within ±0.001 degrees

    #print(f"Lat dist: {abs(final_lat - target_lat)}, Lon dist: {abs(final_lon - target_lon)}")

    return [
        0.1 - abs(final_lat - target_lat),
        0.1 - abs(final_lon - target_lon)
    ]




# Define nonlinear constraint
nonlinear_constraint = scipy.optimize.NonlinearConstraint(
    lambda x: constraint_fxn(x, laun_lat, laun_lon, laun_h, lat_target, lon_target),
    [0, 0],  # lower bounds
    [np.inf, np.inf]  # upper bounds
)

# initial guess
x0 = [750, 60, 20]

#v0, theta, psi bounds
bounds = [(200, 2000), (0, 90),(-180, 180)]

# Testing/verification
"""
thrust_force = 71171545.6    # N
initial_mass = 5000000       # kg (wet mass: fuel + dry)
dry_mass = 500000            # kg (non-burnable structure)
burn_rate = 10000        # kg/s
user_thrust_duration = 300  # s (user-requested)
max_burn_time = (initial_mass - dry_mass) / burn_rate
thrust_duration = min(user_thrust_duration, max_burn_time)

# Projectile properties (mass updated to 10,000 kg)
A = 1.5                 # Cross-sectional area [m^2]
CD0 = 0.295
CD1 = 0.2


laun_lat, laun_lon, laun_h = 48.4158, -101.3584+180, 1878
lat_target,lon_target,h_target= 55.333,83-180,0


optimal_traj = scipy.optimize.minimize(objective_func, x0,method='SLSQP',constraints=[nonlinear_constraint],bounds=bounds)
opt_user_thrust_duration, opt_theta, opt_phi = optimal_traj.x



print(opt_user_thrust_duration, opt_theta, opt_phi)
print(optimal_traj)
"""

Now that we have a function for optimizing an individual trajectory, we robustly determine launch assignments based on the distances from each target to each launch coordinate. Assignments are made based on minimizing the farthest launch, then assigning targets to the closest launch site. This minimizes the slowest launch and each consecutive launch.

In [ ]:

bases = {"Warren":[41.1265,-104.8668+180],
         "Minot":[48.41583,-101.35806+180],
         "Malmstrom":[47.5028,-111.1857+180]}

destinations = {"Dombarovsky": [51.1100, 59.8400-180],
                "Kozelsk": [54.0340, 35.7800-180],
                "Tatishchevo":  [51.6670, 45.1170-180],
                "Uzhur":  [55.3330, 89.8170-180],
                "Teykovo":  [56.8510, 40.5320-180],
                "Yoshkar":  [56.6330, 47.8710-180],
                "Nizhny": [58.0830, 60.0500-180],
                "Novosibirsk": [55.3330, 83.0000-180],
                "Irkutsk":  [52.3170, 104.2330-180]}

site_capacity = 4
site_names = list(bases.keys())
target_names = list(destinations.keys())
Warren_distances = {}
Minot_distances = {}
Malmstrom_distances = {}

for site in destinations.items():
  Warr_dist = 0
  Mino_dist = 0
  Malm_dist = 0
  Warr_dist =  np.sqrt(abs(bases["Warren"][0]-site[1][0])**2 + abs(bases["Warren"][1] - site[1][1])**2)
  Mino_dist =  np.sqrt(abs(bases["Minot"][0]-site[1][0])**2 + abs(bases["Minot"][1] - site[1][1])**2)
  Malm_dist =  np.sqrt(abs(bases["Malmstrom"][0]-site[1][0])**2 + abs(bases["Malmstrom"][1] - site[1][1])**2)
  Warren_distances[site[0]] = Warr_dist
  Minot_distances[site[0]] = Mino_dist
  Malmstrom_distances[site[0]] = Malm_dist

#print(Warren_distances)
#print(Minot_distances)
#print(Malmstrom_distances)

# We now know Minot is farthest from each target,
# Therefore we only give it one target assignment, which will be slowest
min_dist = 1000
minot_targ = ''
for dist in Minot_distances.items():
  curr_dist = dist[1]
  if curr_dist < min_dist:
    min_dist = curr_dist
    minot_targ = dist[0]

Minot_targets = {minot_targ:min_dist}
#print(Minot_targets)

# Because Malmstrom is closer than Warren to each target, we assign
# the farthest targets to Malmstrom to minimize their distances
Warren_targets = {}
Malmstrom_targets = {}

sorted_targets = sorted(Malmstrom_distances.items(), key=lambda item: item[1], reverse=True)

# Get the top 4 items
farthest_targets = sorted_targets[:4]
for target in farthest_targets:
  Malmstrom_targets[target[0]] = target[1]
#print(Malmstrom_targets)

for target in Warren_distances.items():
  if target[0] not in Malmstrom_targets.keys() and  target[0] not in Minot_targets.keys():
    Warren_targets[target[0]] = target[1]

#print(Warren_targets)
#print(Minot_targets)
#print(Malmstrom_targets)

target_assignments = {'Warren':list(Warren_targets.keys()),
                      'Minot':list(Minot_targets.keys()),
                      'Malmstrom':list(Malmstrom_targets.keys())}

print(target_assignments)




With all targets assigned to launch sites, this cell optimizes each launch, storing each initial trajectory parameter.

In [ ]:
Warren_target_cords = [['Dombarovsky',51.1100, 59.8400-180],[ 'Uzhur',55.3330, 89.8170-180],
 ['Nizhny',58.0830, 60.0500-180],['Novosibirsk',55.3330, 83.0000-180]]

Minot_target_cords = [['Irkutsk',52.3170, 104.2330-180]]

Malmstrom_target_cords = [['Kozelsk', 54.0340, 35.7800-180],
 ['Teykovo',56.8510, 40.5320-180],
  ['Tatishchevo',51.6670, 45.1170-180],
   ['Yoshkar',56.6330, 47.8710-180]]

# Constants
thrust_force = 71171545.6    # N
initial_mass = 5000000       # kg (wet mass: fuel + dry)
dry_mass = 500000            # kg (non-burnable structure)
burn_rate = 10000        # kg/s
user_thrust_duration = 300  # s (user-requested)
max_burn_time = (initial_mass - dry_mass) / burn_rate
thrust_duration = min(user_thrust_duration, max_burn_time)
# Projectile properties (mass updated to 10,000 kg)
A = 1.5                 # Cross-sectional area [m^2]
CD0 = 0.295
CD1 = 0.2

print("Beginning optimization of Warren AFB targets...")
for launch in Warren_target_cords:

  laun_lat, laun_lon, laun_h = 41.1265,-104.8668+180,1878
  lat_target,lon_target,h_target= launch[1], launch[2], 0

  optimal_traj = scipy.optimize.minimize(objective_func, x0,method='SLSQP',constraints=[nonlinear_constraint],bounds=bounds)
  opt_v, opt_theta, opt_psi = optimal_traj.x

  v_E = opt_v * np.cos(np.radians(opt_theta)) * np.sin(np.radians(opt_psi))
  v_N = opt_v * np.cos(np.radians(opt_theta)) * np.cos(np.radians(opt_psi))
  v_U = opt_v * np.sin(np.radians(opt_theta))
  v_enu = [v_E,v_N,v_U]
  print(f"{launch[0]}: v,theta,psi: {opt_v},{opt_theta},{opt_psi} // v_enu: {v_enu}")
  launch.append([opt_v,opt_theta,opt_psi, v_enu])

  print(optimal_traj)

print("Roger that Warren AFB, all target trajectories optimized.")

print("Beginning optimization of Minot AFB targets...")

for launch in Minot_target_cords:

  laun_lat, laun_lon, laun_h = 48.41583,-101.35806+180,508.4
  lat_target,lon_target,h_target= launch[1],launch[2],0

  optimal_traj = scipy.optimize.minimize(objective_func, x0,method='SLSQP',constraints=[nonlinear_constraint],bounds=bounds)
  opt_v, opt_theta, opt_psi = optimal_traj.x

  v_E = opt_v * np.cos(np.radians(opt_theta)) * np.sin(np.radians(opt_psi))
  v_N = opt_v * np.cos(np.radians(opt_theta)) * np.cos(np.radians(opt_psi))
  v_U = opt_v * np.sin(np.radians(opt_theta))
  v_enu = [v_E,v_N,v_U]
  print(f"{launch[0]}: v,theta,psi: {opt_v},{opt_theta},{opt_psi} // v_enu: {v_enu}")
  launch.append([opt_v,opt_theta,opt_psi, v_enu])

  print(optimal_traj)

print("Roger that Minot AFB, all target trajectories optimized.")


print("Beginning optimization of Malmstrom AFB targets...")
for launch in Malmstrom_target_cords:

  laun_lat, laun_lon, laun_h = 47.5028,-111.1857+180,1058
  lat_target,lon_target,h_target= launch[1],launch[2],0

  optimal_traj = scipy.optimize.minimize(objective_func, x0,method='SLSQP',constraints=[nonlinear_constraint],bounds=bounds)
  opt_v, opt_theta, opt_psi = optimal_traj.x

  v_E = opt_v * np.cos(np.radians(opt_theta)) * np.sin(np.radians(opt_psi))
  v_N = opt_v * np.cos(np.radians(opt_theta)) * np.cos(np.radians(opt_psi))
  v_U = opt_v * np.sin(np.radians(opt_theta))
  v_enu = [v_E,v_N,v_U]
  print(f"{launch[0]}: v,theta,psi: {opt_v},{opt_theta},{opt_psi} // v_enu: {v_enu}")
  launch.append([opt_v,opt_theta,opt_psi, v_enu])

  print(optimal_traj)

print("Roger that Malmstrom AFB, all target trajectories optimized. ")


print("All targets accounted for, Operation AidDelivery Part 2 completed. ")
print("Trajectory parameters by launch assignment:  ")

print(Warren_target_cords)
print(Minot_target_cords)
print(Malmstrom_target_cords)


| Launch initial trajectory parameters |

# **Warren:**


Dombarovsky: v, theta, psi: 1470.7964689373719, 51.70453674414701, 2.226049326039018

Uzhur: v,theta, psi: 1480.2737970435849, 53.13222626592452, -15.281600255427604

Nizhny: v, theta, psi: 1387.81029779333, 54.506812853580136, 1.4627832710563273

Novosibirsk: v, theta, psi: 1474.9012575876632, 52.937700607391164, -11.482883801450054



# **Minot:**


Irkutsk: v, theta, psi: 1419.5750688952508, 55.53082441863052, -22.713505359496885


# **Malmstrom:**

Kozelsk: v, theta, psi: 1287.4515875953507, 56.86773916893211, 12.9959920398184

Teykovo: v, theta, psi: 1257.336099427747, 57.81683074725379, 9.471513206170464

Tatishchevo: v, theta, psi: 1344.5095648005688, 55.20841331863136, 7.876384180162651

Yoshkar: v, theta, psi: 1289.2062697743763, 57.35280674012376, 5.33304857483101




This cell simulates and visualizes each launch trajectory.

In [ ]:
bases = {
    "Warren": [41.1265, 75.1332],      # -104.8668 + 180
    "Minot": [48.41583, 78.64194],     # -101.35806 + 180
    "Malmstrom": [47.5028, 68.8143],   # -111.1857 + 180
}

# Target vectors and assignments
venus = {
    "Dombarovsky": [35.4037, 910.7896, 1154.3185],
    "Uzhur": [-234.0759, 856.7183, 1184.252],
    "Nizhny": [20.5694, 805.5086, 1129.9337],
    "Novosibirsk": [-176.9575, 871.106, 1176.9427],
    "Irkutsk": [-310.2219, 741.1186, 1170.3414],
    "Kozelsk": [158.2472, 685.6626, 1078.1262],
    "Teykovo": [110.2027, 660.5625, 1064.146],
    "Tatishchevo": [105.1298, 759.9303, 1104.1556],
    "Yoshkar": [64.6414, 692.4705, 1085.5224],
}

launch_assignments = {
    "Dombarovsky": "Warren",
    "Uzhur": "Warren",
    "Nizhny": "Warren",
    "Novosibirsk": "Warren",
    "Irkutsk": "Minot",
    "Kozelsk": "Malmstrom",
    "Teykovo": "Malmstrom",
    "Tatishchevo": "Malmstrom",
    "Yoshkar": "Malmstrom",
}

# Load Earth texture
img = Image.open("earth_texture.jpg")
img_array = np.asarray(img)
img_array = np.hstack([img_array, img_array[:, :1, ...]])
surfacecolor = img_array[:, :, :3].mean(axis=2)

u = np.linspace(0, 2 * np.pi, surfacecolor.shape[1], endpoint=False)
v = np.linspace(0, np.pi, surfacecolor.shape[0])
U, V = np.meshgrid(u, v)
x_sphere = REarth * np.cos(U) * np.sin(V)
y_sphere = REarth * np.sin(U) * np.sin(V)
z_sphere = REarth * np.cos(V)

fig = go.Figure()

fig.add_trace(go.Surface(
    x=x_sphere, y=y_sphere, z=z_sphere,
    surfacecolor=surfacecolor,
    cmin=0, cmax=255,
    colorscale='gray',
    showscale=False,
    opacity=1.0
))

for target, v_enu in venus.items():
    base = launch_assignments[target]
    lat, lon = bases[base]
    sol = simulate_trajectory(lat, lon, 0, v_enu)
    r = sol['y']
    x_traj, y_traj, z_traj = r[0], r[1], r[2]
    print(r[:,0])
    lat_launch, lon_launch, _ = ecef_to_geodetic(r[:3, 0])
    lat_impact, lon_impact, _ = ecef_to_geodetic(r[:3, -1])
    lon_launch = np.deg2rad(lon_launch)
    lon_impact = np.deg2rad(lon_impact)
    lat_launch = np.deg2rad(lat_launch)
    lat_impact = np.deg2rad(lat_impact)

    launch_x = REarth * np.cos(lat_launch) * np.cos(lon_launch)
    launch_y = REarth * np.cos(lat_launch) * np.sin(lon_launch)
    launch_z = REarth * np.sin(lat_launch)

    impact_x = REarth * np.cos(lat_impact) * np.cos(lon_impact)
    impact_y = REarth * np.cos(lat_impact) * np.sin(lon_impact)
    impact_z = REarth * np.sin(lat_impact)

    fig.add_trace(go.Scatter3d(
        x=x_traj, y=y_traj, z=z_traj,
        mode='lines',
        line=dict(width=3),
        name=f"{base} → {target}"
    ))

    fig.add_trace(go.Scatter3d(
        x=[impact_x], y=[impact_y], z=[impact_z],
        mode='markers',
        marker=dict(size=5, color='red'),
        text=[f'{target}'],
        hoverinfo='text',
        showlegend=False

    ))
    fig.add_trace(go.Scatter3d(
      x=[launch_x], y=[launch_y], z=[launch_z],
      mode='markers',
      marker=dict(size=6, color='green'),
      text=[f'{base}'],
      hoverinfo='text',
      showlegend=False
    ))

fig.update_layout(
    title='Optimized rocket trajectories',
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='data'
    ),
    width=1000,
    height=900
)

fig.show()

# **Part 3: Generating Machine Learning Test Cases**

In this section, the code used to generate the data / training cases for the machine learning model is shown. The rocket parameters and properties are the same values as Part 1. To generate the cases, a function is used to randomly generate the inputs: east and north velocities between -1,000 and 1,000 m/s, upwards velocities between 0 and 1,000 m/s, latitudes between -90 and 90, longitudes between -180 and 180, and altitudes between 0 and 20,000 meters. Unfortuneatly, more extreme parameters were not able to be tested due to their computional cost. This function was used to generate 2,000 inputs, which was deemed a sufficient amount to build the machine learning model off of. The simulator in Part 1 was then used to determine the landing locations of all 2,000 input cases, and was then combined into a CSV file with the six aforementioned inputs.

**NOTE**: this code is not meant to be run; generating the training data takes over an hour long. In order to run the machine learning code, please upload the trajectory_training_dataset.csv file. If you do wish to run this code, uncomment the lines that say “# UNCOMMENT LINE TO RUN PROGRAM”.


In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import math
import random
import pandas as pd

thrust_force = 71171545.6    # N
initial_mass = 5000000       # kg (wet mass: fuel + dry)
dry_mass = 500000            # kg (non-burnable structure)
burn_rate = 10000        # kg/s
user_thrust_duration = 300  # s (user-requested)
max_burn_time = (initial_mass - dry_mass) / burn_rate
thrust_duration = min(user_thrust_duration, max_burn_time)
# Projectile properties (mass updated to 10,000 kg)
A = 1.5                 # Cross-sectional area [m^2]
CD0 = 0.295
CD1 = 0.2


# Projectile properties (mass updated to 10,000 kg)
A = 1.5                 # Cross-sectional area [m^2]
CD0 = 0.295
CD1 = 0.2

# PARAMETERS / INPUTS

def generate_test_data(num_samples=10000, seed=40):
    np.random.seed(seed)

    # Random latitude [-90, 90]
    latitudes = np.random.uniform(-90, 90, num_samples)

    # Random longitude [-180, 180]
    longitudes = np.random.uniform(-180, 180, num_samples)

    # Altitude [0, 20000] meters
    altitudes = np.random.uniform(0, 50000, num_samples)

    # Velocity components
    veast = np.random.uniform(-2500, 2500, num_samples)
    vnorth = np.random.uniform(-2500, 2500, num_samples)
    vup = np.random.uniform(0, 2500, num_samples)

    # Combine into DataFrame
    data = pd.DataFrame({
        'latitude': latitudes,
        'longitude': longitudes,
        'altitude': altitudes,
        'veast': veast,
        'vnorth': vnorth,
        'vup': vup,
    })

    # Add velocity vector column for ML input
    data['testvenu'] = data[['veast', 'vnorth', 'vup']].values.tolist()

    return data


# Generate and preview
#testinputs = generate_test_data(2000) # UNCOMMENT LINE TO RUN PROGRAM
print(testinputs.head())

# CODE FOR OUTPUTS

testoutputs = []

for i, row in testinputs.iterrows():
    # Simulate trajectory — assumes it returns a result object with .y shape (3, N)
    #result = simulate_trajectory( # UNCOMMENT LINE TO RUN PROGRAM
        row['latitude'],
        row['longitude'],
        row['altitude'],
        row['testvenu']
    )

    # Convert ECEF output to geodetic
    lat_deg, lon_deg, h = ecef_to_geodetic(result.y[:3, -1])
    lon_deg = (lon_deg - 180) % 360

    # Store geodetic output
    testoutputs.append({
        'latitude_out': lat_deg,
        'longitude_out': lon_deg,
        'altitude_out': h
    })

# Convert to DataFrame
testoutputs_df = pd.DataFrame(testoutputs)
print(testoutputs_df.head())

# Test to make sure everything works
solution2 = simulate_trajectory(45.0, 90.0, 0.0, [100.0, 0.0, 500.0])
lat_deg, lon_deg, h = ecef_to_geodetic(solution2.y[:3, -1])
lon_deg = (lon_deg - 180) % 360
print("Impact geodetic coordinates (lat, lon, alt):", lat_deg, lon_deg, h)



**Save Training Data for testing and analyzing**

In [ ]:
full_data = pd.concat([testinputs[['latitude', 'longitude', 'altitude', 'veast', 'vnorth', 'vup']], testoutputs_df[['latitude_out', 'longitude_out', 'altitude_out']]], axis=1)

# Save to CSV
full_data.to_csv('trajectory_training_dataset.csv', index=False)

print("Saved to trajectory_training_dataset.csv")

# **Part 3: XGBOOST MACHINE LEARNING MODEL**

For the actual machine learning model, we decided a decision tree based approach would be a good method to attempt. We decided to test Random Forest Regression and XGBoost Regression to see which would yield the best results. Firstly, we filtered the training data CSV file; some of the cases went to space due to starting at higher velocities at high altitudes, which left us with 1956 cases out of the original 2000. Then, the regression models were run, and their mean squared error and percent error based on a Haversine function were calculated. After testing different numbers of estimators and maximum tree depths, we received the least error with an XGBoost regressor with 100 estimators and a max depth of 18. This gave us a mean squared error of 884.5 and median percent error of roughly 4.4%. The mean error was being dragged up by a few outliers, which were printed out below the percent error summary.

This first cell block filters the training data csv and defines the input and output features of the dataframe. After the parameters for the XGBoost parameters are set, the model is then trained based off 90% of the training data, and the remaining 10% is used for testing / results comparison. The mean squared error and head of the results dataframe, comparing the actual vs predicted locations, are printed.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor


# Load and prepare your filtered data
trajectory_df = pd.read_csv('trajectory_training_dataset.csv')

# Filter rows (as you already did)
trajectory_df = trajectory_df[trajectory_df['altitude_out'] <= 1]

# Define input (X) and output (y) features
X = trajectory_df[['latitude', 'longitude', 'altitude', 'veast', 'vnorth', 'vup']]
y = trajectory_df[['latitude_out', 'longitude_out']]  # 2D target


# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Create and train the Random Forest Regressor
model = XGBRegressor(objective='reg:squarederror', tree_method='hist', n_estimators=100, random_state=42, max_depth=18)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate performance
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

# Optional: Look at the first few predictions vs actual
results = pd.DataFrame({
    'latitude_out_actual': y_test['latitude_out'].values,
    'longitude_out_actual': y_test['longitude_out'].values,
    'latitude_out_pred': y_pred[:, 0],
    'longitude_out_pred': y_pred[:, 1]
})

print(results.head())

###############################################################################


**Percent Error Calculations with Haversine:**

Here the percent error of each launch is calculated, by finding the Haversine distance from the launch point to the predicted and actual landing spots and determining how off the predicted result was. A summary of the percent error findings is outputted, which shows a median error of ~4.4%.

In [ ]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    """
    Compute great-circle distance between (lat1, lon1) and (lat2, lon2)
    in meters using the Haversine formula.
    Accepts scalar or NumPy array inputs.
    """
    R = 6371000  # Earth radius in meters
    lat1 = np.radians(lat1)
    lat2 = np.radians(lat2)
    dlat = lat2 - lat1
    dlon = np.radians(lon2 - lon1)

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c  # Distance in meters


# Get predicted values from model
y_pred = model.predict(X_test)

# Convert all series to numpy arrays (important!)
launch_lat = X_test['latitude'].values
launch_lon = X_test['longitude'].values
actual_lat = y_test['latitude_out'].values
actual_lon = y_test['longitude_out'].values
pred_lat = y_pred[:, 0]
pred_lon = y_pred[:, 1]

# Step 1: Distance between predicted and actual landing points
pred_vs_actual_dist = haversine(actual_lat, actual_lon, pred_lat, pred_lon)

# Step 2: Distance from launch site to actual landing
launch_to_actual_dist = haversine(launch_lat, launch_lon, actual_lat, actual_lon)

# Avoid divide-by-zero
launch_to_actual_dist = np.where(launch_to_actual_dist == 0, 1e-6, launch_to_actual_dist)

# Step 3: Compute percent error
percent_error = 100 * pred_vs_actual_dist / launch_to_actual_dist


results['percent_error'] = percent_error
print(results[['percent_error']].describe())

**Outlier Analysis:**

This cell just examines the outliers as determined by the percent error calculations from Haversine. We wanted to really try and analyze where the errors were coming from. Many were because of high velocity and altitude cases not having the most data to go off of, while some were because of shorter flights leading to a higher percent error despite relatively low actual differences.

In [ ]:

#print(results.head())
#print(results.tail())

############################3

# Reset indices to ensure alignment
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
results = results.reset_index(drop=True)

# Identify high-error rows
high_error_indices = results[results['percent_error'] > 50].index
high_error_inputs = X_test.loc[high_error_indices]

# Build a summary DataFrame
high_error_summary = high_error_inputs.copy()
high_error_summary['latitude_out_actual'] = results.loc[high_error_indices, 'latitude_out_actual'].values
high_error_summary['longitude_out_actual'] = results.loc[high_error_indices, 'longitude_out_actual'].values
high_error_summary['latitude_out_pred'] = results.loc[high_error_indices, 'latitude_out_pred'].values
high_error_summary['longitude_out_pred'] = results.loc[high_error_indices, 'longitude_out_pred'].values
high_error_summary['percent_error'] = results.loc[high_error_indices, 'percent_error'].values

# Display sorted by percent error
print(high_error_summary[['latitude', 'longitude', 'altitude', 'veast', 'vnorth', 'vup',
                          'latitude_out_actual', 'longitude_out_actual',
                          'latitude_out_pred', 'longitude_out_pred',
                          'percent_error']].sort_values(by='percent_error', ascending=False))

# Issues right now:
# Terms getting sent to space (filtered out)
# Latitudes over +- 90, Longitudes over +- 180 ???

**Validation:**
Here, some of the trajectories in Part 2 were validated by plugging them into the machine learning model. Unfortunately, because many of these cases were higher velocity cases and took paths that pushed the boundaries of the coordinate system data, the machine learning model did not predict very accurate outcomes.

In [ ]:
# --- Specific test case ---
test_case = pd.DataFrame({
    'latitude': [41.1265],
    'longitude': [-104.8668],
    'altitude': [0.0],
    'veast': [35.40371190526211],
    'vnorth': [910.7895770233815],
    'vup': [1154.318490109008]
})

# Predict output geodetic position
predicted_output = model.predict(test_case)

pred_lat, pred_lon = predicted_output[0]
print(f"Predicted landing coordinates, Warren to Dombarovsky (lat, lon): ({pred_lat:.6f}, {pred_lon:.6f})")

##################################
test_case = pd.DataFrame({
    'latitude': [41.1265],
    'longitude': [-104.8668],
    'altitude': [0.0],
    'veast': [-234.0759221505452],
    'vnorth': [856.7183149397758],
    'vup': [1184.2519604079005]
})

# Predict output geodetic position
predicted_output = model.predict(test_case)
pred_lat, pred_lon = predicted_output[0]

print(f"Predicted landing coordinates, Warren to Uzhur (lat, lon): ({pred_lat:.6f}, {pred_lon:.6f})")


##################################
test_case = pd.DataFrame({
    'latitude': [41.1265],
    'longitude': [-104.8668],
    'altitude': [0.0],
    'veast': [20.569413483695342],
    'vnorth': [805.508604092855],
    'vup': [1129.9337195705455]
})

# Predict output geodetic position
predicted_output = model.predict(test_case)

pred_lat, pred_lon = predicted_output[0]
#pred_lon = (pred_lon - 180) % 360
print(f"Predicted landing coordinates, Warren to Nizhny (lat, lon): ({pred_lat:.6f}, {pred_lon:.6f})")

##################################
test_case = pd.DataFrame({
    'latitude': [41.1265],
    'longitude': [-104.8668],
    'altitude': [0.0],
    'veast': [-176.9575339267163],
    'vnorth': [871.1059967371776],
    'vup': [1176.9426890332313]
})

# Predict output geodetic position
predicted_output = model.predict(test_case)

pred_lat, pred_lon = predicted_output[0]
print(f"Predicted landing coordinates, Warren to Novosibirsk (lat, lon): ({pred_lat:.6f}, {pred_lon:.6f})")

# **Part 4: Radar Coverage and Ballistic Projectile Detection Modeling**

Part 4 implements a complete detection-and-prediction pipeline for a ballistic delivery trajectory:


**Trajectory Simulation**

  Convert the launch latitude, longitude, and altitude to ECEF coordinates.

  Transform the initial ENU velocity vector to ECEF and numerically integrate the full equations of motion (gravity, drag, Coriolis, centrifugal and thrust accelerations) until ground impact.


**Radar Detection with Horizon Compensation**
  For each radar site, compute the slant range from the radar to every point on the trajectory.
  Convert each trajectory point back to geodetic coordinates and compute the great-circle ground distance using the Haversine formula.
  Compensate for Earth's curvature by calculating the horizon range at the radar height and at each projectile altitude:

  HorizonRange(h) = √(2·Rₑ·h + h²).

  Identify the first time step where slant < Rmax and ground distance ≤ (horizon₍radar₎ + horizon₍projectile₎).


**Machine-Learning–Based Impact Prediction**

At the detection instant, extract the geodetic state (latitude, longitude, altitude) and local ENU velocity components.

Assemble these six features into a single-row DataFrame and invoke the pre-trained XGBoost regressor.

Measure the model’s inference latency using a high-precision timer, then record the predicted landing latitude and longitude.


**Results Aggregation and Formatting**

Collect per-radar detection metrics (detection time, slant/ground distances, horizon totals), ML latency, predicted landing coordinates, and constant radar parameters into a Pandas DataFrame.

Round and reorder fields for clarity and save the table to CSV for reporting or further analysis.


**Interactive 3D Visualization**

Render an earth-textured globe with a 180°-rotated equirectangular map to ensure correct geographic orientation.

Overlay the full projectile trajectory, launch point, true impact, ML-predicted impact, radar coverage cones, and the designated target marker in a Plotly Scene.


This workflow seamlessly integrates high-fidelity physics, line-of-sight geometry, and machine-learning inference to provide real-time detection and landing-site estimation for humanitarian ballistic delivery missions.

**Part 4 Section 1: Targeting Novosibirsk (Novosibirsk Oblast)**

In [ ]:
# ── Part 4: Radar Coverage with Horizon Compensation & ML‐Based Landing Prediction ──

import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.spatial.transform import Rotation as Rot
import plotly.graph_objects as go
from PIL import Image
import time
from IPython.display import display

# ↳ Assumes Parts 1 & 3 definitions are already in scope:
#    REarth, geodetic_to_ecef, ecef_to_geodetic,
#    ecef_to_enu, simulate_trajectory,
#    model            # your trained XGBRegressor

# ── 1) Reset thrust & mass parameters ─────────────────────────────────────────
thrust_force = 71171545.6    # N
initial_mass = 5000000       # kg (wet mass: fuel + dry)
dry_mass = 500000            # kg (non-burnable structure)
burn_rate = 10000        # kg/s
user_thrust_duration = 300  # s (user-requested)
max_burn_time = (initial_mass - dry_mass) / burn_rate
thrust_duration = min(user_thrust_duration, max_burn_time)
# Projectile properties (mass updated to 10,000 kg)
A = 1.5                 # Cross-sectional area [m^2]
CD0 = 0.295
CD1 = 0.2

# ── 2) Radar sites & parameters ─────────────────────────────────────────────
radar_sites = {
    "Norilsk": {"lat": 69.3558, "lon":  88.1893},
    "Yakutsk": {"lat": 62.0355, "lon": 129.6755}
}
Rmax   = 500e3       # detection range [m]
alpha  = 360         # azimuth coverage [deg]
beta   = 90          # elevation coverage [deg]
radar_h= 10.0        # radar elevation [m]

# ── 3) Helper functions ─────────────────────────────────────────────────────
def horizon_range(h):
    v = 2*REarth*h + h*h
    return np.sqrt(np.maximum(v, 0.0))

def haversine(lat1, lon1, lat2, lon2):
    R = 6_371_000.0
    φ1, φ2 = np.radians(lat1), np.radians(lat2)
    dφ = φ2 - φ1
    dλ = np.radians(lon2 - lon1)
    a = np.sin(dφ/2)**2 + np.cos(φ1)*np.cos(φ2)*np.sin(dλ/2)**2
    return R * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

# ── 4) Simulate trajectory ──────────────────────────────────────────────────
launch_lat, launch_lon = 48.4158, -101.3584
v_enu = [-124.181441142, 756.020748663, 1135.95558396]
sol = simulate_trajectory(launch_lat, launch_lon, 0.0, v_enu)

# ── 5) Detection, classification & ML prediction (measure ML latency) ───────
t    = sol.t
pos  = sol.y[:3, :].T      # shape (N,3)
vels = sol.y[3:6, :].T     # shape (N,3)

records = []
for name, info in radar_sites.items():
    # slant & ground distances
    radar_ecef = geodetic_to_ecef(info["lat"], info["lon"], 0.0)
    slant = np.linalg.norm(pos - radar_ecef, axis=1)
    geo   = np.array([ecef_to_geodetic(p) for p in pos])
    lats, lons, alts = geo[:,0], geo[:,1], geo[:,2]
    ground = np.array([
        haversine(info["lat"], info["lon"], φ, λ)
        for φ,λ in zip(lats, lons)
    ])
    hr = horizon_range(radar_h)
    hp = horizon_range(alts)
    total_hz = hr + hp

    idx = np.where((slant < Rmax) & (ground <= total_hz))[0]
    rec = {"Radar": name, "Detected": bool(idx.size)}

    if idx.size:
        i0    = idx[0]
        t_det = t[i0]
        # classification point immediately at detection
        lat_c, lon_c, alt_c = ecef_to_geodetic(pos[i0])
        vecef = vels[i0]
        ve, vn, vu = ecef_to_enu(vecef, lat_c, lon_c)
        print(lat_c, lon_c, alt_c, ve, vn, vu)

        # ML‐predict landing and measure latency
        test_case = pd.DataFrame({
            'latitude':  [lat_c],
            'longitude': [lon_c],
            'altitude':  [alt_c],
            'veast':     [ve],
            'vnorth':    [vn],
            'vup':       [vu]
        })
        start = time.perf_counter()
        pred_lat, pred_lon = model.predict(test_case)[0]
        ml_latency = time.perf_counter() - start

        rec.update({
            "Detect Time (s)":     t_det,
            "Slant Range (m)":     slant[i0],
            "Ground Dist (m)":     ground[i0],
            "Horizon Total (m)":   total_hz[i0],
            "ML Latency (s)":      ml_latency,
            "Pred Land Lat (deg)": pred_lat,
            "Pred Land Lon (deg)": pred_lon-180
        })
    else:
        # fill NaNs if not detected
        for key in ["Detect Time (s)", "Slant Range (m)", "Ground Dist (m)",
                    "Horizon Total (m)", "ML Latency (s)",
                    "Pred Land Lat (deg)", "Pred Land Lon (deg)"]:
            rec[key] = np.nan

    # constants
    rec["Radar Area (m^2)"]   = np.pi * Rmax**2
    rec["Radar Volume (m^3)"] = (4/3)*np.pi * Rmax**3 * (alpha/360)*(beta/360)
    rec["Radar Horizon (m)"]  = horizon_range(radar_h)

    records.append(rec)

# Build the main results DataFrame
df_results = pd.DataFrame(records)

# Reorder & round for display
cols_main = [
    "Radar", "Detected",
    "Detect Time (s)", "ML Latency (s)",
    "Slant Range (m)", "Ground Dist (m)", "Horizon Total (m)",
    "Pred Land Lat (deg)", "Pred Land Lon (deg)",
    "Radar Area (m^2)", "Radar Volume (m^3)", "Radar Horizon (m)"
]
df_results = df_results[cols_main]
df_results[cols_main[2:]] = df_results[cols_main[2:]].round(4)

# Display the original results
print("=== Detection & ML Prediction Results ===")
display(df_results)

# Compute the actual impact point
lat_imp, lon_imp, _ = ecef_to_geodetic(sol.y[:3, -1])

# Build a concise comparison DataFrame
df_compare = df_results[["Radar", "Pred Land Lat (deg)", "Pred Land Lon (deg)"]].copy()
df_compare["Actual Lat (deg)"] = round(lat_imp, 4)
df_compare["Actual Lon (deg)"] = round(lon_imp, 4)

# Display the comparison under the original
print("\n=== Predicted vs. Actual Landing Locations ===")
display(df_compare)

# ── 6) 3D Plot: Earth, trajectory, launch/impact, ML‐predicted impact & cones ──

img = Image.open("earth_texture.jpg")
arr = np.asarray(img)                       # shape (H, W, 3)
# roll by half the width → 180° about NS axis
arr = np.roll(arr, arr.shape[1]//2, axis=1)
# then stitch one column back on to fix the seam
arr = np.hstack([arr, arr[:, :1, :]])
surface = arr.mean(axis=2)

# now build your sphere mesh exactly as before
u = np.linspace(0, 2*np.pi, surface.shape[1], endpoint=False)
v = np.linspace(0,    np.pi, surface.shape[0])
U, V = np.meshgrid(u, v)
x_sph = REarth * np.cos(U) * np.sin(V)
y_sph = REarth * np.sin(U) * np.sin(V)
z_sph = REarth * np.cos(V)

fig = go.Figure()
fig.add_trace(go.Surface(
    x=x_sph, y=y_sph, z=z_sph,
    surfacecolor=surface, cmin=0, cmax=255,
    colorscale='gray', showscale=False
))

# Trajectory
traj = sol.y[:3, :]
fig.add_trace(go.Scatter3d(
    x=traj[0], y=traj[1], z=traj[2],
    mode='lines', line=dict(color='orange', width=4),
    name='Trajectory'
))

# Launch & true impact
launch_ecef = geodetic_to_ecef(launch_lat, launch_lon, 5.0)
lat_imp, lon_imp, _ = ecef_to_geodetic(traj[:,-1])
impact_ecef = geodetic_to_ecef(lat_imp, lon_imp, 5.0)
fig.add_trace(go.Scatter3d(
    x=[launch_ecef[0]], y=[launch_ecef[1]], z=[launch_ecef[2]],
    mode='markers', marker=dict(size=6, color='green'),
    name='Launch'
))
fig.add_trace(go.Scatter3d(
    x=[impact_ecef[0]], y=[impact_ecef[1]], z=[impact_ecef[2]],
    mode='markers', marker=dict(size=6, color='red'),
    name='Impact'
))

if df_results['Detected'].any():
    pred_ecef = geodetic_to_ecef(pred_lat, (pred_lon-180), 5.0)
    fig.add_trace(go.Scatter3d(
        x=[pred_ecef[0]], y=[pred_ecef[1]], z=[pred_ecef[2]],
        mode='markers', marker=dict(size=6, color='yellow'),
        name='ML Predicted'
    ))

# ── Add target marker ───────────────────────────────────────────────────────
target_lat, target_lon = 55.3330, 83.0000
target_ecef = geodetic_to_ecef(target_lat, target_lon, 0.0)
fig.add_trace(go.Scatter3d(
    x=[target_ecef[0]], y=[target_ecef[1]], z=[target_ecef[2]],
    mode='markers',
    marker=dict(size=8, color='cyan'),
    name='Target (55.3330°N, 83.0000°E)'
))

# Radar cones
def add_cone(fig, lat, lon, color='blue'):
    θ = np.linspace(0, 2*np.pi, 30)
    φ = np.linspace(0, np.deg2rad(beta), 15)
    Θ, Φ = np.meshgrid(θ, φ)
    xs = Rmax * np.sin(Φ) * np.cos(Θ)
    ys = Rmax * np.sin(Φ) * np.sin(Θ)
    zs = Rmax * np.cos(Φ)
    base = geodetic_to_ecef(lat, lon, 0)
    up   = base / np.linalg.norm(base)
    axis = np.cross([0,0,1], up)
    if np.linalg.norm(axis)<1e-6:
        Rmat = np.eye(3)
    else:
        ang  = np.arccos(np.dot(up,[0,0,1]))
        Rmat = Rot.from_rotvec(axis/np.linalg.norm(axis)*ang).as_matrix()
    pts_local = np.vstack([xs.ravel(), ys.ravel(), zs.ravel()])
    pts_rot   = Rmat @ pts_local
    pts       = pts_rot.T + base
    fig.add_trace(go.Mesh3d(
        x=pts[:,0], y=pts[:,1], z=pts[:,2],
        alphahull=0, color=color, opacity=0.3,
        name=f"Radar @ {lat:.2f},{lon:.2f}"
    ))

for info in radar_sites.values():
    add_cone(fig, info["lat"], info["lon"])

fig.update_layout(
    title="Part 4: Radar Detection & ML Landing Prediction",
    scene=dict(aspectmode="data"),
    width=900, height=900
)
fig.show()

As seen on the interactive graph, the rocket sucessfully reaches its target, impacting within a few kilometers of its target. The machine learining model was able to accuratly predict the location of the rocket when it impacted the radar detection sphere, but due to its high speed and altitude, combined with training data that did not encompass high speed and altitude trajectories, the ML model was not able to accuratly predict the landing location of the rocket. The LM model reached its conclusion in approximatly 0.0215 seconds, sucessfully classifying the projectile, and reporting on the projectiles interception with the radar sphere. Due to the non-ballistic nature of the projectile due to its added thrust, the model was unable to accuratly predict the landing location of the projectile.

**Part 4 Section 2: Targeting 39.6N, 124.7E**

From Part 2, v_enu = [-621.78885, 704.3224241379983, 1238.3876894568]

In [ ]:
# ── Part 4: Radar Coverage with Horizon Compensation & ML‐Based Landing Prediction ──

import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.spatial.transform import Rotation as Rot
import plotly.graph_objects as go
from PIL import Image
import time
from IPython.display import display

# ↳ Assumes Parts 1 & 3 definitions are already in scope:
#    REarth, geodetic_to_ecef, ecef_to_geodetic,
#    ecef_to_enu, simulate_trajectory,
#    model            # your trained XGBRegressor

# ── 1) Reset thrust & mass parameters ─────────────────────────────────────────
thrust_force         = 71171545.6    # N
initial_mass         = 5_000_000     # kg (wet mass)
dry_mass             =   500_000     # kg (structure)
burn_rate            =     2_500     # kg/s
user_thrust_duration =       435     # s requested
max_burn_time        = (initial_mass - dry_mass) / burn_rate
thrust_duration      = min(user_thrust_duration, max_burn_time)

# ── 2) Radar sites & parameters ─────────────────────────────────────────────
radar_sites = {
    "Norilsk": {"lat": 69.3558, "lon":  88.1893},
    "Yakutsk": {"lat": 62.0355, "lon": 129.6755}
}
Rmax   = 400e3       # detection range [m]
alpha  = 360         # azimuth coverage [deg]
beta   = 90          # elevation coverage [deg]
radar_h= 10.0        # radar elevation [m]

# ── 3) Helper functions ─────────────────────────────────────────────────────
def horizon_range(h):
    v = 2*REarth*h + h*h
    return np.sqrt(np.maximum(v, 0.0))

def haversine(lat1, lon1, lat2, lon2):
    R = 6_371_000.0
    φ1, φ2 = np.radians(lat1), np.radians(lat2)
    dφ = φ2 - φ1
    dλ = np.radians(lon2 - lon1)
    a = np.sin(dφ/2)**2 + np.cos(φ1)*np.cos(φ2)*np.sin(dλ/2)**2
    return R * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

# ── 4) Simulate trajectory ──────────────────────────────────────────────────
launch_lat, launch_lon = 48.4158, -101.3584
v_enu = [-621.78885, 704.3224241379983, 1238.3876894568]
sol = simulate_trajectory(launch_lat, launch_lon, 0.0, v_enu)

# ── 5) Detection, classification & ML prediction (measure ML latency) ───────
t    = sol.t
pos  = sol.y[:3, :].T      # shape (N,3)
vels = sol.y[3:6, :].T     # shape (N,3)

records = []
for name, info in radar_sites.items():
    # slant & ground distances
    radar_ecef = geodetic_to_ecef(info["lat"], info["lon"], 0.0)
    slant = np.linalg.norm(pos - radar_ecef, axis=1)
    geo   = np.array([ecef_to_geodetic(p) for p in pos])
    lats, lons, alts = geo[:,0], geo[:,1], geo[:,2]
    ground = np.array([
        haversine(info["lat"], info["lon"], φ, λ)
        for φ,λ in zip(lats, lons)
    ])
    hr = horizon_range(radar_h)
    hp = horizon_range(alts)
    total_hz = hr + hp

    idx = np.where((slant < Rmax) & (ground <= total_hz))[0]
    rec = {"Radar": name, "Detected": bool(idx.size)}

    if idx.size:
        i0    = idx[0]
        t_det = t[i0]
        # classification point immediately at detection
        lat_c, lon_c, alt_c = ecef_to_geodetic(pos[i0])
        vecef = vels[i0]
        ve, vn, vu = ecef_to_enu(vecef, lat_c, lon_c)

        # ML‐predict landing and measure latency
        test_case = pd.DataFrame({
            'latitude':  [lat_c],
            'longitude': [lon_c],
            'altitude':  [alt_c],
            'veast':     [ve],
            'vnorth':    [vn],
            'vup':       [vu]
        })
        start = time.perf_counter()
        pred_lat, pred_lon = model.predict(test_case)[0]
        ml_latency = time.perf_counter() - start

        rec.update({
            "Detect Time (s)":     t_det,
            "Slant Range (m)":     slant[i0],
            "Ground Dist (m)":     ground[i0],
            "Horizon Total (m)":   total_hz[i0],
            "ML Latency (s)":      ml_latency,
            "Pred Land Lat (deg)": pred_lat,
            "Pred Land Lon (deg)": pred_lon-180
        })
    else:
        # fill NaNs if not detected
        for key in ["Detect Time (s)", "Slant Range (m)", "Ground Dist (m)",
                    "Horizon Total (m)", "ML Latency (s)",
                    "Pred Land Lat (deg)", "Pred Land Lon (deg)"]:
            rec[key] = np.nan

    # constants
    rec["Radar Area (m^2)"]   = np.pi * Rmax**2
    rec["Radar Volume (m^3)"] = (4/3)*np.pi * Rmax**3 * (alpha/360)*(beta/360)
    rec["Radar Horizon (m)"]  = horizon_range(radar_h)

    records.append(rec)

# Build the main results DataFrame
df_results = pd.DataFrame(records)

# Reorder & round for display
cols_main = [
    "Radar", "Detected",
    "Detect Time (s)", "ML Latency (s)",
    "Slant Range (m)", "Ground Dist (m)", "Horizon Total (m)",
    "Pred Land Lat (deg)", "Pred Land Lon (deg)",
    "Radar Area (m^2)", "Radar Volume (m^3)", "Radar Horizon (m)"
]
df_results = df_results[cols_main]
df_results[cols_main[2:]] = df_results[cols_main[2:]].round(4)

# Display the original results
print("=== Detection & ML Prediction Results ===")
display(df_results)

# Compute the actual impact point
lat_imp, lon_imp, _ = ecef_to_geodetic(sol.y[:3, -1])

# Build a concise comparison DataFrame
df_compare = df_results[["Radar", "Pred Land Lat (deg)", "Pred Land Lon (deg)"]].copy()
df_compare["Actual Lat (deg)"] = round(lat_imp, 4)
df_compare["Actual Lon (deg)"] = round(lon_imp, 4)

# Display the comparison under the original
print("\n=== Predicted vs. Actual Landing Locations ===")
display(df_compare)

# ── 6) 3D Plot: Earth, trajectory, launch/impact, ML‐predicted impact & cones ──

img = Image.open("earth_texture.jpg")
arr = np.asarray(img)                       # shape (H, W, 3)
# roll by half the width → 180° about NS axis
arr = np.roll(arr, arr.shape[1]//2, axis=1)
# then stitch one column back on to fix the seam
arr = np.hstack([arr, arr[:, :1, :]])
surface = arr.mean(axis=2)

# now build your sphere mesh exactly as before
u = np.linspace(0, 2*np.pi, surface.shape[1], endpoint=False)
v = np.linspace(0,    np.pi, surface.shape[0])
U, V = np.meshgrid(u, v)
x_sph = REarth * np.cos(U) * np.sin(V)
y_sph = REarth * np.sin(U) * np.sin(V)
z_sph = REarth * np.cos(V)

fig = go.Figure()
fig.add_trace(go.Surface(
    x=x_sph, y=y_sph, z=z_sph,
    surfacecolor=surface, cmin=0, cmax=255,
    colorscale='gray', showscale=False
))

# Trajectory
traj = sol.y[:3, :]
fig.add_trace(go.Scatter3d(
    x=traj[0], y=traj[1], z=traj[2],
    mode='lines', line=dict(color='orange', width=4),
    name='Trajectory'
))

# Launch & true impact
launch_ecef = geodetic_to_ecef(launch_lat, launch_lon, 5.0)
lat_imp, lon_imp, _ = ecef_to_geodetic(traj[:,-1])
impact_ecef = geodetic_to_ecef(lat_imp, lon_imp, 5.0)
fig.add_trace(go.Scatter3d(
    x=[launch_ecef[0]], y=[launch_ecef[1]], z=[launch_ecef[2]],
    mode='markers', marker=dict(size=6, color='green'),
    name='Launch'
))
fig.add_trace(go.Scatter3d(
    x=[impact_ecef[0]], y=[impact_ecef[1]], z=[impact_ecef[2]],
    mode='markers', marker=dict(size=6, color='red'),
    name='Impact'
))

if df_results['Detected'].any():
    pred_ecef = geodetic_to_ecef(pred_lat, (pred_lon-180), 5.0)
    fig.add_trace(go.Scatter3d(
        x=[pred_ecef[0]], y=[pred_ecef[1]], z=[pred_ecef[2]],
        mode='markers', marker=dict(size=6, color='yellow'),
        name='ML Predicted'
    ))
# ── Add target marker ───────────────────────────────────────────────────────
target_lat, target_lon = 39.6, 124.7
target_ecef = geodetic_to_ecef(target_lat, target_lon, 0.0)
fig.add_trace(go.Scatter3d(
    x=[target_ecef[0]], y=[target_ecef[1]], z=[target_ecef[2]],
    mode='markers',
    marker=dict(size=8, color='cyan'),
    name='Target (39.6 °N, 124.7°E)'
))

# Radar cones
def add_cone(fig, lat, lon, color='blue'):
    θ = np.linspace(0, 2*np.pi, 30)
    φ = np.linspace(0, np.deg2rad(beta), 15)
    Θ, Φ = np.meshgrid(θ, φ)
    xs = Rmax * np.sin(Φ) * np.cos(Θ)
    ys = Rmax * np.sin(Φ) * np.sin(Θ)
    zs = Rmax * np.cos(Φ)
    base = geodetic_to_ecef(lat, lon, 0)
    up   = base / np.linalg.norm(base)
    axis = np.cross([0,0,1], up)
    if np.linalg.norm(axis)<1e-6:
        Rmat = np.eye(3)
    else:
        ang  = np.arccos(np.dot(up,[0,0,1]))
        Rmat = Rot.from_rotvec(axis/np.linalg.norm(axis)*ang).as_matrix()
    pts_local = np.vstack([xs.ravel(), ys.ravel(), zs.ravel()])
    pts_rot   = Rmat @ pts_local
    pts       = pts_rot.T + base
    fig.add_trace(go.Mesh3d(
        x=pts[:,0], y=pts[:,1], z=pts[:,2],
        alphahull=0, color=color, opacity=0.3,
        name=f"Radar @ {lat:.2f},{lon:.2f}"
    ))

for info in radar_sites.values():
    add_cone(fig, info["lat"], info["lon"])

fig.update_layout(
    title="Part 4: Radar Detection & ML Landing Prediction",
    scene=dict(aspectmode="data"),
    width=900, height=900
)
fig.show()


As seen on the interactive graph above, the rocket was able to again hit its target, but in this case, never passed through a radar detection sphere, allowing it to go on unnoticed to its target. Therefore, the machine learning model was never used to attempt to predict its final location, as no data was available to it.